# Układy równań liniowych

### Pojęcia warte poznania

* Układ równań liniowych: https://pl.wikipedia.org/wiki/Układ_równań_liniowych
* Rząd macierzy: https://pl.wikipedia.org/wiki/Rząd_macierzy
* Kombinacja liniowa: https://pl.wikipedia.org/wiki/Kombinacja_liniowa
* Eliminacja Gaussa: https://pl.wikipedia.org/wiki/Metoda_eliminacji_Gaussa, Kincaid-Cheney_*_ str. 245, pełny pseudokod: str. 252
* Pivoting: https://en.wikipedia.org/wiki/Pivot_element#Partial_and_complete_pivoting, K.C. str. 261, pełny pseudokod: str. 267
* Norma wektora: https://pl.wikipedia.org/wiki/Przestrze%C5%84_unormowana, K.C. str. 320
* Norma macierzy: https://pl.wikipedia.org/wiki/Norma_macierzowa
* Macierz dodatnio określona: https://pl.wikipedia.org/wiki/Określoność_formy
* Faktoryzacja LU: https://pl.wikipedia.org/wiki/Metoda_LU, K.C. str. 294
* Faktoryzacja Cholesky'ego: https://pl.wikipedia.org/wiki/Rozkład_Choleskiego, K.C. str. 305

Dodatkowo:
* Wskaźnik uwarunkowania: https://pl.wikipedia.org/wiki/Wska%C5%BAnik_uwarunkowania, K.C. str.321
* Metoda Jacobiego: https://en.wikipedia.org/wiki/Jacobi_method, K.C. 323

Książka dla wytrwałych (naprawdę): Y. Saad "Iterative Methods for Sparse Linear Systems"

In [4]:
from typing import Optional, Tuple
import numpy as np

### Zadanie rozgrzewkowe:
Napisać mnożenie macierzy w ulubionym_\**_ języku programowania.

**Pytanko:** jakie muszą być wymiary mnożonych macierzy? (Który wymiar musi się "zgadzać"?)

**Zadanko:** Uzupełnić brakujące wymiary macierzy w docstringu (z dokładnością do ["alfa-konwersji"](https://pl.wikipedia.org/wiki/Konwersja_α))

In [16]:
def agh_superfast_matrix_multiply(a: np.matrix, b: np.matrix) -> np.matrix:
    """Perform totally ordinary multiplication of matrices.
    
    :param a: matrix with dimensions _ by _
    :param b: matrix with dimensions _ by _
    :return:  matrix with dimensions _ by _
    """
    n, k = a.shape
    m = b.shape[1]
    a1 = a.A
    b1 = b.A
    c = np.zeros((n, m), dtype=float)
    for i in range(n):
        for j in range(m):
            for l in range(k):
                c[i][j] += a1[i][l] * b1[l][j]
    return c if not (isinstance(a, np.matrix) and isinstance(b, np.matrix)) else np.matrix(c)

m1 = np.matrix([[1, 2],
                [3, 4],
                [4, 5],
                [5, 1]])

m2 = np.matrix([[1, 2, 3],
                [4, 5, 6]])

res = agh_superfast_matrix_multiply(m1, m2)
assert np.allclose(res, m1 * m2), "Wrong multiplication result"

### Zadania
1. Przeczytać rozdz. 7. Kincaida i Cheney'a (Systems of Linear Equations).
2. Przeczytać rozdz. 8. Kincaida i Cheney'a (Additional Topics Concerning Systems of Linear Equations).
3. Napisać kod (w ulubionym_\**_ języku) do eliminacji Gaussa z i bez pivotingu.
4. Rozwiązać poniższy układ równań z pivotingiem i bez, porównać wyniki:

In [12]:
A = np.matrix([[0.0001, -5.0300, 5.8090, 7.8320],
               [2.2660, 1.9950,  1.2120, 8.0080],
               [8.8500, 5.6810,  4.5520, 1.3020],
               [6.7750, -2.253,  2.9080, 3.9700]])

b = np.matrix([9.5740, 7.2190, 5.7300, 6.2910]).transpose()

x = np.linalg.solve(A, b)

x

matrix([[ 0.21602477],
        [-0.00791511],
        [ 0.63524333],
        [ 0.74617428]])

**Pytanie**: dlaczego wołamy `transpose()` na wektorze `b`?

Sprawdźmy, czy rozwiązanie jest ok (**Pytanie'**: dlaczego po prostu nie użyjemy `==` lub jakiegoś `equals`?):

In [ ]:
np.allclose(np.dot(A, x), b)

In [9]:
def gauss_elimination(A, B, dtype=float):
    a = np.copy(A)
    b = np.copy(B)
    n = a.shape[0]

    for k in range(0, n):
        for i in range(k+1, n):
            xmult = a[i][k] / a[k][k]
            a[i][k] = xmult
            for j in range(k+1, n):
                a[i][j] -= xmult * a[k][j]
            b[i] -= xmult * b[k]

    x = np.zeros((n,1), dtype=dtype)
    x[n-1][0] = b[n-1] / a[n-1][n-1]

    for i in range(n-1, -1, -1):
        summ = b[i]
        for j in range(i+1, n):
            summ -= a[i][j] * x[j][0]
        x[i][0] = summ / a[i][i]

    return np.matrix(x) if isinstance(A, np.matrix) else x


def gauss_with_pivoting(A, B, dtype=float):
    a = np.copy(A)
    b = np.copy(B)
    n = a.shape[0]
    x = np.zeros((n,1), dtype=dtype)
    s = np.zeros(n, dtype=dtype)
    p = np.zeros(n, dtype=int)

    for i in range(n):
        p[i] = i
        smax = 0
        for j in range(n):
            smax = max(smax, abs(a[i][j]))
        s[i] = smax

    for k in range(n-1):
        rmax = 0
        j = k
        for i in range(k+1, n):
            r = abs(a[p[i]][k] / s[p[i]])
            if r > rmax:
                rmax = r
                j = i
        p[j], p[k] = p[k], p[j]
        for i in range(k+1, n):
            xmult = a[p[i]][k] / a[p[k]][k]
            a[p[i]][k] = xmult
            for j in range(k+1, n):
                a[p[i]][j] -= xmult * a[p[k]][j]

    for k in range(n):
        for i in range(k+1, n):
            b[p[i]] -= a[p[i]][k] * b[p[k]]

    x[n-1][0] = b[p[n-1]] / a[p[n-1]][n-1]
    for i in range(n-1, -1, -1):
        summ = b[p[i]]
        for j in range(i+1, n):
            summ -= a[p[i]][j] * x[j][0]
        x[i][0] = summ / a[p[i]][i]
    return np.matrix(x) if isinstance(A, np.matrix) else x

In [10]:
gauss_elimination(A.A, b.A)

array([[ 0.21602477],
       [-0.00791511],
       [ 0.63524333],
       [ 0.74617428]])

In [13]:
gauss_with_pivoting(A.A, b.A)

array([[ 0.21602477],
       [-0.00791511],
       [ 0.63524333],
       [ 0.74617428]])

### Zadania, c.d.

5. Zaimplementować algorytm faktoryzacji LU macierzy
6. (*) Zaimplementować funkcję sprawdzającą, czy dana macierz jest symetryczna i dodatnio określona
7. Zaimplementować algorytm faktoryzacji Cholesky'ego macierzy

In [18]:
def agh_superfast_lu(a: np.matrix) -> Optional[Tuple[np.matrix, np.matrix]]:
    """Perform LU decomposition of a matrix.
    
    :param a: _
    :return:  _
    """
    n = a.shape[0]
    a1 = A.A
    l = np.zeros((n, n), dtype=dtype)
    u = np.zeros((n, n), dtype=dtype)
    for k in range(n):
        l[k][k] = 1
        for j in range(k, n):
            u[k][j] = a1[k][j] - sum([l[k][s] * u[s][j] for s in range(k)])
        for i in range(k+1, n):
            l[i][k] = (a1[i][k] - sum([l[i][s] * u[s][k] for s in range(k)])) / u[k][k]
    return Optional((np.matrix(l), np.matrix(u)))


def agh_superfast_check_spd(a: np.matrix) -> bool:
    """Check whether a matrix is symmetric and positive-definite (SPD).
    
    :param a: _
    """
    if a.shape[0] != a.shape[1]:
        return False
    n = a.shape[0]
    a1 = a.A
    for i in range(n):
        for j in range(i,n):
            if a1[i][j] != a1[j][i]:
                return False
    w, v = np.linalg.eig(a1)
    return all([lmbd > 0 for lmbd in w])


def agh_superfast_cholesky(a: np.matrix) -> Optional[np.matrix]:
    """Perform a Cholesky decomposition of a matrix.
    
    :param a: _
    :return:  _
    """
    n = A.shape[0]
    a = A.A
    l = np.zeros((n, n), dtype=dtype)
    for k in range(n):
        summ = 0
        for s in range(k):
            summ += l[k][s] * l[k][s]
        l[k][k] = np.sqrt(a[k][k] - summ)
        for i in range(k+1, n):
            summ = 0
            for s in range(k):
                summ += l[i][s] * l[k][s]
            l[i][k] = (a[i][k] - summ) / l[k][k]
    return Optional(np.matrix(l))

### Zadania, opcjonalnie
5. zaimplementować metodę Jacobiego (iteracyjne rozwiązywanie układu równań liniowych)
6. za pomocą tejże metody rozwiązać powyższy układ równań

\*  wszystkie referencje odnoszą się do [książki](https://wiki.iiet.pl/lib/exe/fetch.php?media=studia:przedmioty:mownit:numerical_mathematics_and_computing.pdf) David Kincaid, Ward Cheney - "Numerical Mathematics and Computing, 6th edition"
\** _ulubiony_ język programowania staramy się pojmować rozsądnie, tj. z wyłączeniem języków pokroju Prologa, języków z [tej listy](https://en.wikipedia.org/wiki/Esoteric_programming_language) oraz Assemblera i PHP. Haskella można używać na własną odpowiedzialność.